In [1]:
#INSTALANDO PACOTES
!pip install spacy
!python -m spacy download pt

    100% |████████████████████████████████| 38.7MB 118.0MB/s 
  Running setup.py install for pt-core-news-sm ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/pt

    You can now load the model via spacy.load('pt')



In [0]:
#IMPORTANDO PACOTES

import spacy #IMPORTANDO PACOTE SPACY
nlp = spacy.load('pt') #SETANDO IDIOMA

import pandas as pd #IMPORTANDO PACOTE PANDA

import nltk #IMPORTANDO PACOTE NLTK

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split #IMPORTANDO SPLIT DE TREINO E TESTE

from sklearn.metrics import f1_score #IMPORTANDO MÉTRICA F1 SCORE

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
#CARREGANDO O DATA FRAME
df = pd.read_csv('https://s3-sa-east-1.amazonaws.com/dados-leonardo/imdb-pt-br.csv', encoding='utf-8')

In [4]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [0]:
df.describe()

,id
count,49459.000000
mean,24730.960917
std,14277.792868
min,1.000000
25%,12366.500000
50%,24731.000000
75%,37095.500000
max,49460.000000


In [5]:
#VERIFICANDO NULOS NO DATASET
df.isna().sum()

id           0
text_en      0
text_pt      0
sentiment    0
dtype: int64

In [0]:
#VETORIZANDO EM TFIDF A COLUNA (TEXT_PT) - UNIGRAMA, BIGRAMA E TRIGRAMA
#OBS: COM STOP WORD (NLTK OU SPACY) A MÉTRICA CAI UM POUCO

vect = TfidfVectorizer(ngram_range=(1,3), use_idf=True, max_df=0.95, min_df=2)
vect.fit(df.text_pt)
text_vect = vect.transform(df.text_pt)

In [0]:
#SEPARANDO OS DADOS EM TREINO (80%) E TESTE (20%)
X_train, X_test, y_train, y_test = train_test_split(text_vect, df.sentiment, test_size=0.20, random_state=42)

In [0]:
#TESTANDO ALGUNS ALGORITMOS PARA IDENTIFICAR QUAL GERA A MELHOR METRICA F1
clfs = {
    'mnb': MultinomialNB(),
    'gnb': BernoulliNB(),
    'rfc': RandomForestClassifier(),
    'lr': LogisticRegression()
}

In [9]:
# TREINANDO OS MODELOS
f1_scores = dict()
for clf_name in clfs:
    clf = clfs[clf_name]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1_scores[clf_name] = f1_score(y_pred, y_test, average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
#VERIFICANDO AS MÉTRICAS F1
f1_scores

{'gnb': 0.887489271235977,
 'lr': 0.8899034077410228,
 'mnb': 0.8929465959794863,
 'rfc': 0.7598551667675277}

In [0]:
## OTIMIZANDO OS DOIS ALGORITMOS COM OS MELHORES RESULTADOS

In [11]:
# AJUSTANDO OS PARÂMETROS DO NAIVE BAYES
nb = MultinomialNB(alpha = 0.2, fit_prior=True)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

print('MELHOR RESULTADO COM NAIVE BAYES: ', f1_score(y_pred, y_test, average='weighted'))

MELHOR RESULTADO COM NAIVE BAYES:  0.8984953267653701


In [12]:
#AJUSTANDO OS PARÂMETROS DA REGRESSÃO LOGÍSTICA
lg = LogisticRegression(penalty = 'l2', C = 100,random_state = 0)
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

print('MELHOR RESULTADO COM REGRESSÃO LOGÍSTICA: ', f1_score(y_pred, y_test, average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


MELHOR RESULTADO COM REGRESSÃO LOGÍSTICA:  0.9111333918783125


In [13]:
#TESTANDO O MODELO COM DADOS ALEATÓRIOS
texto = vect.transform(['O filme é bom.'])
print('Previsão: ', lg.predict(texto))

texto = vect.transform(['O filme é ruim.'])
print('Previsão: ', lg.predict(texto))

texto = vect.transform(['Gostei do filme.'])
print('Previsão: ', lg.predict(texto))

texto = vect.transform(['Não Gostei do filme.'])
print('Previsão: ', lg.predict(texto))

texto = vect.transform(['O final foi decepcionante.'])
print('Previsão: ', lg.predict(texto))

texto = vect.transform(['O final foi alucinante.'])
print('Previsão: ', lg.predict(texto))

Previsão:  ['pos']
Previsão:  ['neg']
Previsão:  ['pos']
Previsão:  ['neg']
Previsão:  ['neg']
Previsão:  ['pos']


In [0]:
### É ISSO GALERA! ESTE MATERIAL É UMA BASE PARA CLASSIFICAÇÃO PARA ANÁLISE DE SENTIMENTOS EM PYTHON UTILIZANDO O PACOTE SKLEARN ! ABS